In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
traffic = pd.read_csv("/content/drive/My Drive/traffic forecast/holt_results.csv")

In [ ]:
from fbprophet import Prophet

In [ ]:
traffic.head()

,Unnamed: 0,index,road_name,timestamp,Total Vehicles,arima Prediction,holt Prediction
0,0,0,Abbotsford Street,2015-08-21 00:00:00,16,NaN,NaN
1,1,1,Abbotsford Street,2015-08-21 01:00:00,28,NaN,NaN
2,2,2,Abbotsford Street,2015-08-21 02:00:00,11,NaN,NaN
3,3,3,Abbotsford Street,2015-08-21 03:00:00,14,NaN,NaN
4,4,4,Abbotsford Street,2015-08-21 04:00:00,33,NaN,NaN


In [ ]:
% cd /content/drive/My Drive/traffic forecast

/content/drive/My Drive/traffic forecast


In [ ]:
prophet_results = pd.DataFrame()
n=0
for  roadname in traffic.road_name.unique():
    road = traffic[traffic.road_name == roadname].reset_index()
    train = road.iloc[:int(0.7*len(road))][['timestamp','Total Vehicles']]
    valid = road.iloc[int(0.7*len(road)):][['timestamp','Total Vehicles']]
    train.columns = ['ds','y']
    valid.columns = ['ds','y']
    model = Prophet()
    model.fit(train)

    forecast = model.predict(valid)['yhat']
    forecast = pd.DataFrame(forecast.values,index = valid.index,columns=['prophet Prediction'])
    prophet_results =  prophet_results.append( road.merge(forecast,how='left',left_index= True , right_index= True))
    prophet_results.to_csv("prophet_results.csv")
    n = n+1
    print(str(n) + " of " + str(len(traffic.road_name.unique())) + str(roadname))

In [ ]:
traffic = pd.read_csv("prophet_results.csv")

In [ ]:
test = traffic[~traffic['prophet Prediction'].isna()]

In [ ]:
test.head()

,Unnamed: 0,level_0,Unnamed: 0.1,index,road_name,timestamp,Total Vehicles,arima Prediction,holt Prediction,prophet Prediction
117,117,117,117,117,Abbotsford Street,2015-08-25 21:00:00,120,98.434844,83.893651,124.139308
118,118,118,118,118,Abbotsford Street,2015-08-25 22:00:00,65,112.705683,104.128198,119.717105
119,119,119,119,119,Abbotsford Street,2015-08-25 23:00:00,42,79.654384,61.596761,120.696612
120,120,120,120,120,Abbotsford Street,2015-08-26 00:00:00,30,56.328629,23.806142,109.136203
121,121,121,121,121,Abbotsford Street,2015-08-26 01:00:00,12,47.979243,2.053875,83.429065


In [ ]:
mean_absolute_error (test['Total Vehicles'],test['prophet Prediction']),r2_score(test['Total Vehicles'],test['prophet Prediction'])

(137.16139787376534, 0.5756545756054869)